In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [2]:
BASE_URL = "https://pengadaan.info/lelang"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"
}

TOTAL_PAGES = 9


In [3]:
def crawl_page(page: int):
    url = f"{BASE_URL}?page={page}"
    print(f"[INFO] Crawling page {page}: {url}")

    resp = requests.get(url, headers=HEADERS, timeout=30)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")

    items = soup.select("div.box-list div.item")
    print(f"[INFO] Found {len(items)} items on page {page}")

    results = []

    for item in items:
        title_tag = item.select_one("h4 a")
        if not title_tag:
            continue

        lpse_tag = item.select_one("h5 span.color-black")
        p_info = item.select_one("p.text-truncate")
        hps_tag = item.select_one("a.btn-success")
        jadwal_tag = item.select_one("a.btn-color-biru")

        metode = satker = tahap = None

        if p_info:
            lines = list(p_info.stripped_strings)

            for line in lines:
                if line.startswith("Metode"):
                    metode = line.replace("Metode :", "").strip()
                elif line.startswith("Satker"):
                    satker = line.replace("Satker :", "").strip()

            tahap_tag = p_info.select_one("span.text-tahap")
            if tahap_tag:
                tahap = tahap_tag.get_text(strip=True)

        results.append({
            "judul": title_tag.get_text(strip=True),
            "url": title_tag.get("href"),
            "lpse": lpse_tag.get_text(strip=True) if lpse_tag else None,
            "metode": metode,
            "satker": satker,
            "tahap": tahap,
            "hps": hps_tag.get_text(strip=True) if hps_tag else None,
            "jadwal": jadwal_tag.get_text(strip=True) if jadwal_tag else None,
            "page": page
        })

    return results


In [4]:
all_data = []

for page in range(1, TOTAL_PAGES + 1):
    try:
        page_data = crawl_page(page)
        all_data.extend(page_data)
        time.sleep(1)  # sopan ke server
    except Exception as e:
        print(f"[ERROR] Page {page}: {e}")


[INFO] Crawling page 1: https://pengadaan.info/lelang?page=1
[INFO] Found 10 items on page 1
[INFO] Crawling page 2: https://pengadaan.info/lelang?page=2
[INFO] Found 10 items on page 2
[INFO] Crawling page 3: https://pengadaan.info/lelang?page=3
[INFO] Found 10 items on page 3
[INFO] Crawling page 4: https://pengadaan.info/lelang?page=4
[INFO] Found 10 items on page 4
[INFO] Crawling page 5: https://pengadaan.info/lelang?page=5
[INFO] Found 10 items on page 5
[INFO] Crawling page 6: https://pengadaan.info/lelang?page=6
[INFO] Found 10 items on page 6
[INFO] Crawling page 7: https://pengadaan.info/lelang?page=7
[INFO] Found 10 items on page 7
[INFO] Crawling page 8: https://pengadaan.info/lelang?page=8
[INFO] Found 10 items on page 8
[INFO] Crawling page 9: https://pengadaan.info/lelang?page=9
[INFO] Found 10 items on page 9


In [5]:
df = pd.DataFrame(all_data)
df.head()


,judul,url,lpse,metode,satker,tahap,hps,jadwal,page
0,Supervisi Konstruksi Pembangunan Pengendalian ...,https://pengadaan.info/detail/supervisi-konstr...,LPSE Kementrian Pekerjaan Umum dan Perumahan R...,-,694201 / SNVT PELAKSANAAN JARINGAN SUMBER AIR ...,Pengumuman Prakualifikasi [...],"Hps Rp 2.000.000.000,00",22 January 2026 14:00:00 s/d 31 March 2026 16:...,1
1,Supervisi Konstruksi Pembangunan Pengendalian ...,https://pengadaan.info/detail/supervisi-konstr...,LPSE Kementrian Pekerjaan Umum dan Perumahan R...,-,694201 / SNVT PELAKSANAAN JARINGAN SUMBER AIR ...,Pengumuman Prakualifikasi [...],"Hps Rp 2.499.996.905,00",22 January 2026 14:00:00 s/d 31 March 2026 16:...,1
2,Supervisi Konstruksi Peningkatan Fungsi Tampun...,https://pengadaan.info/detail/supervisi-konstr...,LPSE Kementrian Pekerjaan Umum dan Perumahan R...,-,694143 / SNVT PEMBANGUNAN BENDUNGAN BWS SULAWE...,Pengumuman Prakualifikasi [...],"Hps Rp 880.000.000,00",22 January 2026 12:00:00 s/d 27 March 2026 16:...,1
3,PERENCANAAN PENANGANAN BENCANA ALAM JALAN NASI...,https://pengadaan.info/detail/perencanaan-pena...,LPSE Kementrian Pekerjaan Umum dan Perumahan R...,-,693839,Pengumuman Prakualifikasi [...],"Hps Rp 2.400.392.000,00",22 January 2026 12:00:00 s/d 31 March 2026 16:...,1
4,Pemutakhiran peta daerah irigasi kewenangan pu...,https://pengadaan.info/detail/pemutakhiran-pet...,LPSE Kementrian Pekerjaan Umum dan Perumahan R...,-,694289,Pengumuman Prakualifikasi [...],"Hps Rp 916.885.000,00",22 January 2026 11:00:00 s/d 31 March 2026 16:...,1


In [6]:
print("Total rows:", len(df))
df.info()


Total rows: 90
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   judul   90 non-null     object
 1   url     90 non-null     object
 2   lpse    90 non-null     object
 3   metode  90 non-null     object
 4   satker  90 non-null     object
 5   tahap   90 non-null     object
 6   hps     90 non-null     object
 7   jadwal  90 non-null     object
 8   page    90 non-null     int64 
dtypes: int64(1), object(8)
memory usage: 6.5+ KB


In [7]:
df.sample(10)


,judul,url,lpse,metode,satker,tahap,hps,jadwal,page
24,Konsultan Supervisi Rekonstruksi Jalan Cipayun...,https://pengadaan.info/detail/konsultan-superv...,LPSE Kota Depok,-,DINAS PEKERJAAN UMUM DAN PENATAAN RUANG,Pengumuman Prakualifikasi [...],"Hps Rp 195.540.000,00",21 January 2026 16:00:00 s/d 20 March 2026 16:...,3
10,Pengadaan Jasa Outsourcing,https://pengadaan.info/detail/pengadaan-jasa-o...,LPSE TNI Angkatan Udara,-,SESKOAU TNI AU,Pengumuman Pascakualifikasi,"Hps Rp 2.200.000.000,00",21 January 2026 22:00:00 s/d 09 February 2026 ...,2
21,Perencanaan Kolam Renang Rano Wangun Sario Manado,https://pengadaan.info/detail/perencanaan-kola...,LPSE Provinsi Sulawesi Utara,-,DINAS PEKERJAAN UMUM DAN PENATAAN RUANG,Pengumuman Prakualifikasi [...],"Hps Rp 200.000.000,00",21 January 2026 16:30:00 s/d 04 March 2026 17:...,3
64,PERBAIKAN RINGAN GEDUNG BRAMMASTA DAN DISPEN,https://pengadaan.info/detail/perbaikan-ringan...,LPSE TNI Angkatan Laut,-,KOARMADA RI TNI AL,Pengumuman Pascakualifikasi [...],"Hps Rp 1.719.375.000,00",21 January 2026 09:15:00 s/d 18 February 2026 ...,7
54,PRESERVASI JALAN DAN JEMBATAN TAMILOW - HAYA -...,https://pengadaan.info/detail/preservasi-jalan...,LPSE Kementrian Pekerjaan Umum dan Perumahan R...,-,693936,Pengumuman Pascakualifikasi [...],"Hps Rp 6.193.282.000,00",21 January 2026 14:00:00 s/d 11 February 2026 ...,6
1,Supervisi Konstruksi Pembangunan Pengendalian ...,https://pengadaan.info/detail/supervisi-konstr...,LPSE Kementrian Pekerjaan Umum dan Perumahan R...,-,694201 / SNVT PELAKSANAAN JARINGAN SUMBER AIR ...,Pengumuman Prakualifikasi [...],"Hps Rp 2.499.996.905,00",22 January 2026 14:00:00 s/d 31 March 2026 16:...,1
84,Perencanaan Peningkatan/Pengaspalan Jalan Land...,https://pengadaan.info/detail/perencanaan-peni...,LPSE Kabupaten Murung Raya,-,DINAS PERUMAHAN RAKYAT DAN KAWASAN PERMUKIMAN ...,Pengumuman Prakualifikasi [...],"Hps Rp 172.000.000,00",20 January 2026 20:00:00 s/d 27 February 2026 ...,9
71,Pengawasan Pembangunan Flat Type 45 (2 lantai)...,https://pengadaan.info/detail/pengawasan-pemba...,LPSE TNI Angkatan Laut,-,MABES TNI AL TNI AL,Pengumuman Prakualifikasi [...],"Hps Rp 476.773.000,00",20 January 2026 23:00:00 s/d 06 March 2026 23:...,8
39,Rekonstruksi Ruas Jalan Widodaren - Karangasem,https://pengadaan.info/detail/rekonstruksi-rua...,LPSE Kabupaten Pemalang,-,Dinas Pekerjaan Umum dan Penataan Ruang,Pengumuman Pascakualifikasi [...],"Hps Rp 2.461.397.354,33",21 January 2026 15:00:00 s/d 12 February 2026 ...,4
8,Konsultan Perencanaan Pembangunan SPKKL Merauke,https://pengadaan.info/detail/konsultan-perenc...,LPSE Badan Keamanan Laut,-,BADAN KEAMANAN LAUT,Pengumuman Prakualifikasi [...],"Hps Rp 1.798.440.870,00",21 January 2026 18:00:00 s/d 06 April 2026 23:...,1


In [8]:
df["judul"] = df["judul"].str.strip()
df["lpse"] = df["lpse"].str.strip()
df["satker"] = df["satker"].str.strip()
df["tahap"] = df["tahap"].str.strip()

df.head()


,judul,url,lpse,metode,satker,tahap,hps,jadwal,page
0,Supervisi Konstruksi Pembangunan Pengendalian ...,https://pengadaan.info/detail/supervisi-konstr...,LPSE Kementrian Pekerjaan Umum dan Perumahan R...,-,694201 / SNVT PELAKSANAAN JARINGAN SUMBER AIR ...,Pengumuman Prakualifikasi [...],"Hps Rp 2.000.000.000,00",22 January 2026 14:00:00 s/d 31 March 2026 16:...,1
1,Supervisi Konstruksi Pembangunan Pengendalian ...,https://pengadaan.info/detail/supervisi-konstr...,LPSE Kementrian Pekerjaan Umum dan Perumahan R...,-,694201 / SNVT PELAKSANAAN JARINGAN SUMBER AIR ...,Pengumuman Prakualifikasi [...],"Hps Rp 2.499.996.905,00",22 January 2026 14:00:00 s/d 31 March 2026 16:...,1
2,Supervisi Konstruksi Peningkatan Fungsi Tampun...,https://pengadaan.info/detail/supervisi-konstr...,LPSE Kementrian Pekerjaan Umum dan Perumahan R...,-,694143 / SNVT PEMBANGUNAN BENDUNGAN BWS SULAWE...,Pengumuman Prakualifikasi [...],"Hps Rp 880.000.000,00",22 January 2026 12:00:00 s/d 27 March 2026 16:...,1
3,PERENCANAAN PENANGANAN BENCANA ALAM JALAN NASI...,https://pengadaan.info/detail/perencanaan-pena...,LPSE Kementrian Pekerjaan Umum dan Perumahan R...,-,693839,Pengumuman Prakualifikasi [...],"Hps Rp 2.400.392.000,00",22 January 2026 12:00:00 s/d 31 March 2026 16:...,1
4,Pemutakhiran peta daerah irigasi kewenangan pu...,https://pengadaan.info/detail/pemutakhiran-pet...,LPSE Kementrian Pekerjaan Umum dan Perumahan R...,-,694289,Pengumuman Prakualifikasi [...],"Hps Rp 916.885.000,00",22 January 2026 11:00:00 s/d 31 March 2026 16:...,1


In [9]:
df.to_csv("pengadaan_lelang_page1_9.csv", index=False)
print("Saved: pengadaan_lelang_page1_9.csv")


Saved: pengadaan_lelang_page1_9.csv
